<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Meeting-Calendars" data-toc-modified-id="Meeting-Calendars-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Meeting Calendars</a></span></li><li><span><a href="#Minutes-and-Statements" data-toc-modified-id="Minutes-and-Statements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Minutes and Statements</a></span></li></ul></div>

In [1]:
import numpy as np
import requests
from lxml import etree
import calendar
import functools
import re
import json
import os

In [129]:
header = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,zh-TW;q=0.7',
    'DNT': '1',
    'Host': 'www.federalreserve.gov',
    'Referer': 'https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': "Windows",
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1'
}

### Meeting Calendars

In [155]:
monthdict = {month: index for index, month in enumerate(calendar.month_name) if month}
days = []

In [156]:
for year in range(1994,2011):
    base_url = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{}.htm".format(year)
    res = requests.get(url=base_url, headers=header).text
    tree = etree.HTML(res)
    historical_meeting = tree.xpath('//div[@class="panel-heading"]/h5/text()')
    for meeting in historical_meeting:
        if 'Conference Call' in meeting:
            continue
        meeting = meeting.replace('Meeting ','').replace('-',' ') 
        year = meeting.split()[-1]
        date = meeting.split()[-2]
        if len(date) < 2:
            date = '0' + date
        if len(meeting.split()) < 5:
            month = meeting.split()[0]
        else:
            month = meeting.split()[2]
        if month in monthdict:
            month = monthdict[month]
            if month < 10:
                month = '0' + str(month)
            else:
                month = str(month)
        day = year + month + date
#         print(day)
        days.append(day)

In [159]:
for year in range(2011,2016):
    base_url = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{}.htm".format(year)
    res = requests.get(url=base_url, headers=header).text
    tree = etree.HTML(res)
    historical_meeting = tree.xpath('//h5[@class="panel-heading panel-heading--shaded"]/text()')
    for meeting in historical_meeting:
        if 'Conference Call' in meeting:
            continue
        meeting = meeting.replace('Meeting ','').replace('-',' ') 
        year = meeting.split()[-1]
        date = meeting.split()[-2]
        if len(date) < 2:
            date = '0' + date
        if len(meeting.split()) < 5:
            month = meeting.split()[0]
        else:
            month = meeting.split()[2]
        if month in monthdict:
            month = monthdict[month]
            if month < 10:
                month = '0' + str(month)
            else:
                month = str(month)
        day = year + month + date
#         print(day)
        days.append(day)

In [161]:
base_url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
res = requests.get(url=base_url, headers=header).text
tree = etree.HTML(res)
historical_meeting = tree.xpath('//div[@class="col-xs-12 col-md-4 col-lg-4 fomc-meeting__minutes"]/a[2]/@href')
for meeting in historical_meeting:
    day = meeting[27:35]
    days.append(day)

In [165]:
with open('./data/calendars_minutes.txt', 'w') as f:
    for i in range(len(days)):
        s = (re.sub(r"['{ },]*", '', str(days[i])) + '\n').replace(':', ',')
        f.write(s)